<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/%E6%97%A5%E8%8B%B1%E5%AF%BE%E8%A8%B3%E3%83%87%E3%83%BC%E3%82%BF%E3%82%92%E4%BD%9C%E6%88%90%E3%81%99%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ダウンロード

In [1]:
import json
import urllib.parse
import requests
import time

def main(target, output_file_path, debug=False):

    unit = 1000

    endpoint = "https://jpsearch.go.jp/rdf/sparql/" if target == "js" else "https://ld.cultural.jp/sparql"

    output_file_path = f"{output_file_path}_{target}.json"

    page = 0
    data = {}

    start_all = time.time()

    try:
        while 1:
            start = time.time()

            q = f"""PREFIX jps: <https://jpsearch.go.jp/term/property#>
        PREFIX schema: <http://schema.org/>
        PREFIX chname: <https://jpsearch.go.jp/entity/chname/>
        SELECT DISTINCT ?cho ?label ?name WHERE {{
            ?cho rdfs:label ?label;
                schema:name ?name . 
            MINUS {{ ?cho jps:sourceInfo ?s . }}
            filter (lang(?name) = "en")  .
            filter (?label != str(?name)) . 
        }} LIMIT {unit} OFFSET {unit * page}"""

            url = endpoint+"?query="+urllib.parse.quote(q)+"&format=json&output=json&results=json"

            if debug and page == 0:
                print("url", url)

            results = requests.get(url).json()
            if len(results["results"]["bindings"]) == 0:
                break

            page += 1

            if debug:
                print("page", page, "経過時間", time.time() - start)

            for i in range(len(results["results"]["bindings"])):
                
                obj = results["results"]["bindings"][i]

                label = obj["label"]["value"].strip()
                name = obj["name"]["value"].strip()

                if label == name:
                    # continue
                    pass

                data[label] = name

                # cho = obj["cho"]["value"]

                # print(cho, label, name)

            if page > 5:
                # break
                pass

    except Exception as e:
        with open(f"{output_file_path}_{page}.json", 'w') as f:
            json.dump(data, f, ensure_ascii=False, indent=4,
            sort_keys=True, separators=(',', ': '))

    with open(output_file_path, 'w') as f:
        json.dump(data, f, ensure_ascii=False, indent=4,
        sort_keys=True, separators=(',', ': '))

    print("経過時間のtotal", time.time() - start_all)

In [ ]:
# パラメータ

debug = True
# debug = False

output_file_path = "lang.json"

main("js", output_file_path, True)

url https://jpsearch.go.jp/rdf/sparql/?query=PREFIX%20jps%3A%20%3Chttps%3A//jpsearch.go.jp/term/property%23%3E%0A%20%20%20%20%20%20%20%20PREFIX%20schema%3A%20%3Chttp%3A//schema.org/%3E%0A%20%20%20%20%20%20%20%20PREFIX%20chname%3A%20%3Chttps%3A//jpsearch.go.jp/entity/chname/%3E%0A%20%20%20%20%20%20%20%20SELECT%20DISTINCT%20%3Fcho%20%3Flabel%20%3Fname%20WHERE%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%3Fcho%20rdfs%3Alabel%20%3Flabel%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20schema%3Aname%20%3Fname%20.%20%0A%20%20%20%20%20%20%20%20%20%20%20%20MINUS%20%7B%20%3Fcho%20jps%3AsourceInfo%20%3Fs%20.%20%7D%0A%20%20%20%20%20%20%20%20%20%20%20%20filter%20%28lang%28%3Fname%29%20%3D%20%22en%22%29%20%20.%0A%20%20%20%20%20%20%20%20%20%20%20%20filter%20%28%3Flabel%20%21%3D%20str%28%3Fname%29%29%20.%20%0A%20%20%20%20%20%20%20%20%7D%20LIMIT%201000%20OFFSET%200&format=json&output=json&results=json
page 1 経過時間 1.7499938011169434
page 2 経過時間 2.0290324687957764
page 3 経過時間 0.616133451461792
page 